In [1]:
suppressMessages(library(data.table))
suppressMessages(library(tidyverse))
suppressMessages(library(stringr))
suppressMessages(library(ggplot2))
suppressMessages(library(ggpubr))
suppressMessages(library(ggalluvial))
suppressMessages(library(ggsci))
options(warn=-1) # use options(warn=0) To turn warnings back on
options(dplyr.summarise.inform = FALSE)
source("../src/confidence_interval.r")

In [2]:
setwd("/Users/inamojun/TMDU-LR_isoform_atlas/Figures")

In [ ]:
#load data
load("../data/data_SupFig08.RData")

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
g = ss %>%
  # dplyr::filter((PARS_score_renatured+PARS_score_rep1+PARS_score_rep2)/3 < as.numeric(quantile((merged$PARS_score_renatured+merged$PARS_score_rep1+merged$PARS_score_rep2)/3, probs = seq(0, 1, 0.01))["99%"]) &
  #                (PARS_score_renatured+PARS_score_rep1+PARS_score_rep2)/3 > as.numeric(quantile((merged$PARS_score_renatured+merged$PARS_score_rep1+merged$PARS_score_rep2)/3, probs = seq(0, 1, 0.01))["1%"])) %>%
  dplyr::mutate(PARS_score_bin = cut((PARS_score_renatured+PARS_score_rep1+PARS_score_rep2)/3, breaks=c(seq(-7,7,0.1))) %>% stringr::str_split(.,pattern=",",simplify = TRUE) %>% .[,2] %>% gsub("\\]","",.) %>% as.numeric(),
                folding_strength = -1*MFE,
                region = case_when(
                  scaled_distance_from_cap_TSS2TTS <=100 ~ "5'UTR",
                  scaled_distance_from_cap_TSS2TTS <=200 & scaled_distance_from_cap_TSS2TTS > 100 ~ "CDS",
                  scaled_distance_from_cap_TSS2TTS <=300 & scaled_distance_from_cap_TSS2TTS > 200 ~ "3'UTR"
                ),
                region = factor(region, levels=c("5'UTR","CDS","3'UTR"))) %>%
  dplyr::group_by(PARS_score_bin) %>%
  dplyr::summarize(folding_strength_mean = mean(folding_strength),
                   folding_strength_CI_up = confidence_interval_upper(folding_strength),
                   folding_strength_CI_low = confidence_interval_lower(folding_strength)
  ) %>%
  as.data.frame() %>%
  na.omit() %>%
  ggscatter(., x = "PARS_score_bin", y = "folding_strength_mean",
            color = "black", shape = 21, size = 5, # Points color, shape and size
            add = "reg.line",  # Add regressin line
            add.params = list(color = "blue", fill = "lightgray"), # Customize reg. line
            conf.int = TRUE, # Add confidence interval
            cor.coef = TRUE, # Add correlation coefficient. see ?stat_cor
            cor.coeff.args = list(method = "spearman", label.sep = "\n"), cor.coef.size = 6
  ) +
  ggtitle("local folding strength and PARS score") +
  ylab(paste0("folding strength (-MFE)")) +
  xlab("averaged PARS score\n[PARS_score_renatured+PARS_score_rep1+PARS_score_rep2]") +
  theme_minimal() +
  theme(strip.text.x=element_text(size=20, color="black", face="bold"),
        strip.text.y=element_text(size=20, color="black", face="bold"),
        legend.position = "bottom",
        plot.title = element_text(size=20),
        axis.title.x = element_text(size=20),
        axis.title.y = element_text(size =20),
        axis.text.y = element_text(size = 20),
        axis.text.x = element_text(size = 20),
        legend.text =  element_text(size = 20), 
        legend.key.size = grid::unit(0.8, "lines"),
        legend.title = element_text(size = 0, hjust = 0))
g
pdf("./FigS8.pdf", width=6, height=6)
g
dev.off()